# Database Connection & Service Testing
Validates the `Japlayer` SQLite connection and tests Data Service functionality.

### Import dependencies and namespaces
Loads the required EF Core packages and local project DLLs.

In [ ]:
#r "nuget: Microsoft.EntityFrameworkCore.Sqlite"
#r "..\Japlayer.Data\bin\Debug\net8.0\Japlayer.Data.dll"
#r "..\Japlayer.WinUI\bin\Debug\net8.0-windows10.0.19041.0\Japlayer.dll"

using Japlayer.Data.Context;
using Japlayer.Data.Services;
using Japlayer.Models;
using Microsoft.EntityFrameworkCore;
using System.IO;
using System.Text.Json;

### Define settings path
Sets the file path for the local application configuration.

In [ ]:
var settingsFilePath = @"..\japlayer.settings.json.local";

### Initialize Database Context
Parses the settings file and configures the `DatabaseContext` using the SQLite provider.

In [ ]:
string jsonContent = File.ReadAllText(settingsFilePath);
var options = new JsonSerializerOptions { PropertyNameCaseInsensitive = true };
AppSettings settings = JsonSerializer.Deserialize<AppSettings>(jsonContent, options);

settings.Display();

var optionsBuilder = new DbContextOptionsBuilder<DatabaseContext>();
optionsBuilder.UseSqlite($"Data Source={settings.SqliteDatabasePath}");

var context = new DatabaseContext(optionsBuilder.Options);
Console.WriteLine($"✓ Initialized context for: {settings.SqliteDatabasePath}");

### Verify connectivity and record counts
Tests the database connection and retrieves the total number of records for each primary table.

In [ ]:
var canConnect = await context.Database.CanConnectAsync();
Console.WriteLine($"Connection test: {(canConnect ? "SUCCESS" : "FAILED")}\n");

var counts = new []
{
    new { Table = "Media", Count = await context.Media.CountAsync() },
    new { Table = "Metadata", Count = await context.MediaMetadata.CountAsync() },
    new { Table = "Locations", Count = await context.MediaLocations.CountAsync() },
    new { Table = "Images", Count = await context.MediaImages.CountAsync() },
    new { Table = "Genres", Count = await context.MediaGenres.CountAsync() },
    new { Table = "People", Count = await context.MediaPeople.CountAsync() },
    new { Table = "Series", Count = await context.MediaSeries.CountAsync() },
    new { Table = "Studios", Count = await context.MediaStudios.CountAsync() },
    new { Table = "UserTags", Count = await context.UserTags.CountAsync() }
};

counts

### Test Eager Loading
Fetches the first media record while including all related entity collections.

In [ ]:
var firstMedia = await context.Media
    .Include(m => m.MediaMetadata)
    .Include(m => m.MediaImages)
    .Include(m => m.Genres)
    .Include(m => m.People)
    .Include(m => m.Studios)
    .Include(m => m.Series)
    .FirstOrDefaultAsync();

firstMedia

### Query Media Metadata
Retrieves a list of metadata items including their associated cover and thumbnail paths.

In [ ]:
var mediaWithMetadata = await context.MediaMetadata
        .Include(m => m.Media)
        .Include(m => m.CoverNavigation)
        .Include(m => m.ThumbnailNavigation)
        .ToListAsync();

mediaWithMetadata

### Query Media Locations
Fetches all storage location entries from the database.

In [ ]:
var locations = await context.MediaLocations.ToListAsync();
locations

### Instantiate Data Providers
Initializes the high-level service providers using the active database context.

In [ ]:
var mediaProvider = new DatabaseMediaProvider(context);
var sceneProvider = new DatabaseMediaSceneProvider(context);
var imageProvider = new DatabaseImageProvider(context);

### Test Library Provider
Calls the service to retrieve all library items.

In [ ]:
var libraryItems = await mediaProvider.GetLibraryItemsAsync();
libraryItems

### Test specific item retrieval
Filters the library for the first valid item and fetches its detailed record.

In [ ]:
var firstItem = libraryItems.FirstOrDefault(item => !string.IsNullOrEmpty(item.Title));
firstItem

### Test Media Item Service
Fetches a full `MediaItem` object by its unique identifier.

In [ ]:
var mediaItem = await mediaProvider.GetMediaItemAsync(firstItem.MediaId);
mediaItem

### Test Scene Service
Retrieves all defined scenes associated with the current media item.

In [ ]:
var scenes = (await sceneProvider.GetMediaScenesAsync(firstItem.MediaId)).ToList();
scenes

### Test Image Service
Retrieves the file paths for the gallery images associated with the item.

In [ ]:
var galleryPaths = (await imageProvider.GetGalleryPathsAsync(firstItem.MediaId)).ToList();
galleryPaths